<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/boeing_poc_agentai_predict_maintance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai -q
!pip install faiss-cpu -q
!pip install tiktoken -q
!pip install colab_env -q
!pip install --upgrade tiktoken -q

In [27]:
import random
import openai
import faiss
import numpy as np

import colab_env  # This is needed for Google Colab
import os

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


class Boeing787:
    """Represents a Boeing 787 with detailed components and data."""

    def __init__(self, id):
        self.id = id
        self.engine1_health = 100
        self.engine2_health = 100
        self.hydraulic_pressure = 100
        self.fuel_level = 33384  # Gallons, based on Boeing 787-8 capacity
        self.tire_pressure = 208  # PSI, average tire pressure
        self.brake_wear = 0  # Percentage wear
        self.apu_health = 100  # Auxiliary Power Unit health
        self.oxygen_levels = 100  # Cabin oxygen levels
        self.flight_hours = 0
        self.last_maintenance = 0
        self.flight_history = []
        self.maintenance_issues = []
        self.years_in_service = 5  # Added years in service
        self.apu_starts = 0
        self.cabin_pressure_cycles = 0
        self.landing_gear_cycles = 0

    def simulate_flight(self, origin, destination):
        """Simulates a flight with detailed data changes and potential issues."""
        flight_distance = self.get_flight_distance(
            origin, destination
        )  # Get distance
        flight_duration = flight_distance / 560  # Assuming average speed of 560 mph

        self.engine1_health -= random.randint(
            1, 5
        ) + (
            0.1 * flight_distance / 1000
        )  # More wear on longer flights
        self.engine2_health -= random.randint(1, 5) + (
            0.1 * flight_distance / 1000
        )
        self.hydraulic_pressure -= random.randint(0, 3)
        self.fuel_level -= flight_distance * 0.15  # Fuel consumption based on distance
        self.tire_pressure -= (
            random.uniform(0, 1) * (flight_distance / 1000)
        )  # More wear on tires
        self.brake_wear += random.randint(1, 3) * (
            flight_duration / 5
        )  # Brake wear based on flight duration
        self.apu_health -= random.randint(0, 2)
        self.oxygen_levels -= random.uniform(0, 0.5)

        # Simulate additional data points
        self.apu_starts += random.randint(1, 3)  # Count APU starts
        self.cabin_pressure_cycles += 1  # Count pressurization cycles
        self.landing_gear_cycles += 1  # Count landing gear cycles

        self.flight_hours += flight_duration

        # Simulate potential maintenance issues
        if random.random() < 0.1:
            affected_engine = random.choice(["engine1", "engine2"])
            self.maintenance_issues.append(f"{affected_engine} needs inspection")
        if random.random() < 0.05:
            self.maintenance_issues.append("hydraulic system leak")
        if random.random() < 0.02:
            self.maintenance_issues.append("brake system wear")
        if random.random() < 0.01:
            self.maintenance_issues.append("apu malfunction")

        self.flight_history.append(
            {
                "origin": origin,
                "destination": destination,
                "engine1_health": self.engine1_health,
                "engine2_health": self.engine2_health,
                "hydraulic_pressure": self.hydraulic_pressure,
                "fuel_level": self.fuel_level,
                "tire_pressure": self.tire_pressure,
                "brake_wear": self.brake_wear,
                "apu_health": self.apu_health,
                "oxygen_levels": self.oxygen_levels,
                "apu_starts": self.apu_starts,
                "cabin_pressure_cycles": self.cabin_pressure_cycles,
                "landing_gear_cycles": self.landing_gear_cycles,
            }
        )

    def get_flight_distance(self, origin, destination):
        """Retrieves flight distance between two airports."""
        distances = {
            ("Montreal", "Seattle"): 2284,  # Approximately 2284 miles
            ("Vancouver", "Montreal"): 2520,  # Approximately 2520 miles
            # Add more distances as needed
        }
        return distances.get((origin, destination), 2000)  # Default 2000 miles if not found


class PredictiveMaintenanceAgent:
    """An agent that uses GPT-4 with FAISS and CAG for predictive maintenance."""

    def __init__(self, aircraft):
        self.aircraft = aircraft
        self.index = faiss.IndexFlatL2(1536)  # Index for embeddings
        self.prompt_history = []
        self.cache = {}  # Cache to store past prompts and their associated KV cache

    def check_health(self):
        """Checks the health using GPT-4 with FAISS and CAG."""

        # Create a structured flight history table for the prompt
        flight_history_table = "| Origin | Destination | Engine1 Health | Engine2 Health | Hydraulic Pressure | Fuel Level | Tire Pressure | Brake Wear | APU Health | Oxygen Levels | APU Starts | Cabin Pressure Cycles | Landing Gear Cycles |\n"
        flight_history_table += "|---|---|---|---|---|---|---|---|---|---|---|---|---|\n"
        for flight in self.aircraft.flight_history:
            flight_history_table += f"| {flight['origin']} | {flight['destination']} | {flight['engine1_health']} | {flight['engine2_health']} | {flight['hydraulic_pressure']} | {flight['fuel_level']} | {flight['tire_pressure']} | {flight['brake_wear']} | {flight['apu_health']} | {flight['oxygen_levels']} | {flight['apu_starts']} | {flight['cabin_pressure_cycles']} | {flight['landing_gear_cycles']} |\n"

        prompt = f"""
        You are an expert Boeing 787 maintenance advisor.
        Analyze the data and predict potential maintenance needs.

        Aircraft ID: {self.aircraft.id}
        Location: Montreal
        Years in Service: {self.aircraft.years_in_service}
        Flight History:
        {flight_history_table}
        Current Engine1 Health: {self.aircraft.engine1_health}
        Current Engine2 Health: {self.aircraft.engine2_health}
        Current Hydraulic Pressure: {self.aircraft.hydraulic_pressure}
        Current Fuel Level: {self.aircraft.fuel_level}
        Current Tire Pressure: {self.aircraft.tire_pressure}
        Current Brake Wear: {self.aircraft.brake_wear}
        Current APU Health: {self.aircraft.apu_health}
        Current Oxygen Levels: {self.aircraft.oxygen_levels}
        Current APU Starts: {self.aircraft.apu_starts}
        Current Cabin Pressure Cycles: {self.aircraft.cabin_pressure_cycles}
        Current Landing Gear Cycles: {self.aircraft.landing_gear_cycles}
        Flight Hours since last maintenance: {self.aircraft.flight_hours - self.aircraft.last_maintenance}
        Known Issues: {self.aircraft.maintenance_issues}

        Considerations:
        * Values are on a scale of 0 to 100, lower indicates potential issues.
        * Significant drops or consistent downward trends may require maintenance.
        * Consider flight hours, known issues, origin/destination for flight-specific needs, and the aircraft's current location.
        * Fuel level is in gallons. Tire pressure is in PSI. Brake wear is in percentage.
        * The aircraft is currently on the ground in Montreal.
        * The aircraft has been in service for {self.aircraft.years_in_service} years.
        * Hydraulic pressure below 80 requires maintenance.
        * Tire pressure below 60 PSI requires refilling.
        * APU health below 50 or more than 5 starts within 24 hours might affect ground operations.
        * Oxygen levels below 90% are critical.

        Examples:
        * Engine health consistently below 40 after several flights might need maintenance.
        * Sudden hydraulic pressure drop could indicate a leak.
        * Tire pressure below 60 might need refilling.
        * Fuel level should be sufficient for the next flight.
        * Brake wear above 80% requires replacement.
        * APU health below 50 or frequent APU starts might affect ground operations.
        * Oxygen levels below 90% are critical.
        * High cabin pressure cycles might require fuselage inspection.
        * 'engine1 needs inspection' requires immediate attention.
        * Older aircraft might require more frequent inspections.

        Prediction:
        1. **Is maintenance needed?** (yes/no)
        2. **If yes, which component(s)?** (engine1, engine2, hydraulic system, tires, brakes, apu, refuel, oxygen system, fuselage)
        3. **Explanation for your prediction.**
        """

        # Create an embedding for the current prompt
        embedding = self.create_embedding(prompt)

        # Normalize the embedding (for better FAISS performance)
        embedding /= np.linalg.norm(embedding)

        # Search for similar prompts in the history using FAISS
        D, I = self.index.search(embedding.reshape(1, -1), 1)

        response = None  # Initialize response variable

        if D[0][0] < 0.1:  # Adjust similarity threshold as needed
            print("Using similar case from history...")
            similar_prompt_index = I[0][0]
            cached_data = self.prompt_history[similar_prompt_index]
            past_prompt = cached_data[1]  # Retrieve the past prompt

            # Attempt to use cached response (replace with actual CAG implementation)
            if past_prompt in self.cache:
                print("Using cached response...")
                # ... (retrieve and use cached key-value pairs here) ...
                # For now, generate a new prediction since CAG is not implemented
                print("Generating new prediction (CAG not implemented)...")
                response = client.chat.completions.create(
                    model="gpt-4",
                    messages=[
                        {"role": "system", "content": "You are a helpful AI assistant."},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.2  # Lower temperature for more deterministic output
                )

        # If no similar prompt was found or if the cache was empty, generate a new prediction
        if response is None:
            print("Generating new prediction...")
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful AI assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2  # Lower temperature for more deterministic output
            )

        prediction = response.choices[0].message.content.strip()

        # Store the prompt, prediction, and KV cache in the history
        if prompt not in self.cache:
            self.prompt_history.append((embedding, prompt, prediction))
            self.index.add(embedding.reshape(1, -1))
            # self.cache[prompt] = response.key_values  # This is where you would store key-values for CAG

        print(f"GPT-4 (with FAISS and CAG) Prediction: {prediction}")
        self.interpret_prediction(prediction)

    def create_embedding(self, text):
        """Creates an embedding for the given text."""
        response = client.embeddings.create(
            input=text,
            model="text-embedding-ada-002"
        )
        return np.array(response.data[0].embedding)

    def interpret_prediction(self, prediction):
        """Interprets the prediction and takes action."""

        # Split the prediction into lines
        lines = prediction.split("\n")

        # Extract the "Is maintenance needed?" line (case-insensitive)
        maintenance_needed_line = next(
            (line for line in lines if "is maintenance needed?" in line.lower()), None
        )
        if maintenance_needed_line:
            # Check if the line contains "yes" (case-insensitive)
            maintenance_needed = "yes" in maintenance_needed_line.lower()

            if maintenance_needed:
                # Extract the component(s) line (case-insensitive)
                components_line = next(
                    (
                        line
                        for line in lines
                        if "which component(s)?" in line.lower()
                    ),
                    None,
                )
                if components_line:
                    # Split the line by colon if it exists, otherwise split by question mark
                    if ":" in components_line:
                        components = components_line.split(":")[1].strip().lower()
                    else:  # Assuming the format is "which component(s)? brakes"
                        components = components_line.split("?")[1].strip().lower()

                    actions = []
                    if "engine1" in components:
                        actions.append("engine1 maintenance")
                    if "engine2" in components:
                        actions.append("engine2 maintenance")
                    if "hydraulic" in components:
                        actions.append("hydraulic system maintenance")
                    if "tire" in components:
                        actions.append("check tire pressure")
                    if "brake" in components:
                        actions.append("replace brakes")
                    if "apu" in components:
                        actions.append("apu maintenance")
                    if "refuel" in components:
                        actions.append("refuel the aircraft")
                    if "oxygen" in components:
                        actions.append("check oxygen system")
                    self.recommend_maintenance(actions)
                else:
                    print(
                        "Could not parse the prediction (missing component(s) line)."
                    )
            else:
                print("No maintenance needed according to GPT-4.")
        else:
            print(
                "Could not parse the prediction (missing maintenance needed line)."
            )

    def recommend_maintenance(self, actions):
        """Simulates taking maintenance actions."""
        print(f"Aircraft {self.aircraft.id}: Recommended actions - {', '.join(actions)}")
        # Simulate maintenance actions (more detailed)
        if "engine1 maintenance" in actions:
            self.aircraft.engine1_health = min(
                100, self.aircraft.engine1_health + 50
            )
            if "engine1 needs inspection" in self.aircraft.maintenance_issues:
                self.aircraft.maintenance_issues.remove(
                    "engine1 needs inspection"
                )
        if "engine2 maintenance" in actions:
            self.aircraft.engine2_health = min(
                100, self.aircraft.engine2_health + 50
            )
            if "engine2 needs inspection" in self.aircraft.maintenance_issues:
                self.aircraft.maintenance_issues.remove(
                    "engine2 needs inspection"
                )
        if "hydraulic system maintenance" in actions:
            self.aircraft.hydraulic_pressure = min(
                100, self.aircraft.hydraulic_pressure + 50
            )
            if "hydraulic system leak" in self.aircraft.maintenance_issues:
                self.aircraft.maintenance_issues.remove(
                    "hydraulic system leak"
                )
        if "check tire pressure" in actions:
            self.aircraft.tire_pressure = 208  # Refill to normal pressure
        if "replace brakes" in actions:
            self.aircraft.brake_wear = 0
            if "brake system wear" in self.aircraft.maintenance_issues:
                self.aircraft.maintenance_issues.remove("brake system wear")
        if "apu maintenance" in actions:
            self.aircraft.apu_health = min(100, self.aircraft.apu_health + 50)
            if "apu malfunction" in self.aircraft.maintenance_issues:
                self.aircraft.maintenance_issues.remove("apu malfunction")
        if "refuel the aircraft" in actions:
            self.aircraft.fuel_level = 33384
        if "check oxygen system" in actions:
            self.aircraft.oxygen_levels = 100
        self.aircraft.last_maintenance = self.aircraft.flight_hours


# Example usage with Montreal to Seattle flight simulation:
aircraft1 = Boeing787("BA787-1")
agent1 = PredictiveMaintenanceAgent(aircraft1)

# Simulate the last flight from Vancouver to Montreal
aircraft1.simulate_flight("Vancouver", "Montreal")
agent1.check_health()  # Check health after arriving in Montreal

Generating new prediction...
GPT-4 (with FAISS and CAG) Prediction: 1. **Is maintenance needed?** Yes
2. **If yes, which component(s)?** Brakes
3. **Explanation for your prediction:** The brake system wear is listed as a known issue and the brake wear is at 2.7%. Although this is not above the 80% threshold that typically requires replacement, the fact that it is listed as a known issue suggests that there may be other factors at play causing the brakes to wear more quickly or perform suboptimally. Therefore, it would be prudent to conduct a thorough inspection and maintenance of the brake system to ensure safety and performance. All other components are within normal ranges and do not require immediate attention.
Aircraft BA787-1: Recommended actions - replace brakes
